In [1]:
'''데이터 생성 스크립트'''
# bboxes = data['bboxes'] # nested array: [[x, y, w, h],...]
# scores = data['scores'] # [0.6, 0.89,...]
# labels = data['labels'] # [0, 4,...] # yolo 라벨 0-23
# timestamp = data['timestamp'] # unix
# width = data['width'] # 이미지 width
# height = data['height'] # 이미지 height
# region_and_name = data['region_and_name'] # 카메라 정보. id?
# img_directory = data['img_directory'] # 이미지 저장 경로
# dict_values([[[580.052604675293, 217.65965223312378, 60.063819885253906, 58.98589611053467], [330.85792541503906, 175.94241857528687, 33.85063171386719, 39.26307678222656]], [0.6402683258056641, 0.5894748568534851], [0, 4], 1693530106898, 640, 360, '지역', 'C:/cctv_images/지역_1693530106898.jpg'])

'데이터 생성 스크립트'

In [2]:
import pandas as pd

columns = ['cctvId', 'cctvName', 'centerName', 'timestamp', 'xCenter', 'yCenter', 'bWidth', 'bHeight', 'iWidth', 'iHeight', 'score', 'label', 'imgName']

df = pd.DataFrame(columns=columns)
df

,cctvId,cctvName,centerName,timestamp,xCenter,yCenter,bWidth,bHeight,iWidth,iHeight,score,label,imgName


In [3]:
cctv_df = pd.read_csv(r'static\CCTV.csv')
cctv_df.head()

,Unnamed: 0,RN,CCTVID,CCTVNAME,CENTERNAME,XCOORD,YCOORD,URL
0,0,1,L280034,가덕영업소,거제교통정보센터,128.817344,35.025620,https://www.utic.go.kr/view/map/openDataCctvSt...
1,1,2,L280005,거가1교,거제교통정보센터,128.731926,35.017735,https://www.utic.go.kr/view/map/openDataCctvSt...
2,2,3,L280003,거가1교(저도방향),거제교통정보센터,128.721416,35.017084,https://www.utic.go.kr/view/map/openDataCctvSt...
3,3,4,L280054,거가2교,거제교통정보센터,128.758293,35.014669,https://www.utic.go.kr/view/map/openDataCctvSt...
4,4,5,L280001,거제면 귀목정,거제교통정보센터,128.581956,34.862783,https://www.utic.go.kr/view/map/openDataCctvSt...


In [4]:
cctv_df.shape

(7746, 8)

In [29]:
import random
import time
from datetime import datetime, timedelta
import os
class GenerateRandomData:
    def __init__(self, x1=None, y1=None, score=None, label=None, width=None, height=None, cctvid=None, img_directory_pre=None):
        '입력 안 하면 랜덤으로 x1, y1, label 설정됨'
        self._width = width
        self._height = height
        self._generate_width_height()
        self.x1 = x1
        self.y1 = y1
        if not all([self.x1, self.y1]):
            self.x1, self.y1 = self.generate_starting_point()
        self.bbox = self.generate_bbox(self.x1, self.y1)
        self.score = score or self.generate_score()
        self.label = label or self.generate_label()
        self._cctvid = cctvid or self._generate_region_and_name()[0]
        self._img_directory_pre = img_directory_pre or self._generate_img_directory_pre()
        self.img_directory = os.path.join(self._img_directory_pre, self._cctvid + '_' + str(int(time.time()*1000)) + '.jpg')

    def generate_starting_point(self):
        x1 = random.uniform(10, self._width-10)
        y1 = random.uniform(10, self._height-10)
        return x1, y1

    def generate_bbox(self, x, y):
        '''
        x,y coordinates로 bbox 만들기\n
        x, y, bwidth, bheight로 구성
        '''
        bwidth = random.uniform(10,100)
        bheight = random.uniform(10,100)
        # 이미지 사이즈 안 넘어가게
        if x + bwidth/2 > self._width: bwidth = (self._width-x) * 2
        if x - bwidth/2 < 0: bwidth = x * 2
        if y + bheight/2 > self._height: bheight = (self._height-y) * 2
        if y - bheight/2 < 0: bheight = y * 2
        return [x/self._width, y/self._height, bwidth/self._width, bheight/self._height]
    
    def generate_score(self):
        'score 생성'
        return random.uniform(0.5, 1)
    
    def generate_label(self):
        'label 생성'
        return random.randrange(0, 23)
    
    def _generate_width_height(self, new=False):
        'width, height 생성'
        self._width = random.choice([640, 1280, 1600, 1920]) if new else (self._width if self._width else random.choice([640, 1280, 1600, 1920]))
        self._height = int(self._width * (9/16)) if new else (self._height if self._height else int(self._width * (9/16)))
        return self._width, self._height
    
    def generate_timestamp(self):
        'timestamp 생성'
        # 현재 시각을 얻습니다.
        current_time = datetime.now()

        # 무작위로 날짜를 생성합니다. 예: 현재 시각에서 0부터 30일 사이의 무작위 날짜를 생성합니다.
        random_days = random.randint(0, 30)
        random_miliseconds = random.randint(0, 999)
        random_time = current_time + timedelta(days=random_days, milliseconds=random_miliseconds)

        # 무작위로 시, 분, 초를 생성합니다.
        random_hours = random.randint(0, 23)
        random_minutes = random.randint(0, 59)
        random_seconds = random.randint(0, 59)

        # 생성된 시, 분, 초를 반영하여 시간을 설정합니다.
        random_time = random_time.replace(hour=random_hours, minute=random_minutes, second=random_seconds)

        # 무작위 타임스탬프를 생성합니다.
        timestamp = int(random_time.timestamp()*1000)

        return timestamp

    def _generate_region_and_name(self, new=False):
        'region 정보 생성'
        idx = random.randrange(0, 7746)
        cctv_id, cctv_name, center_name = cctv_df.iloc[idx, 2:5]
        return [cctv_id, cctv_name, center_name]
    
    def _generate_img_directory_pre(self, new=False):
        '파일 부모 저장경로 생성'
        return random.choice([r'C:/cctv_images', r'/home/cctv_images', r'~/Downloads/cctv_images'])
    
    def generate_img_directory(self, timestamp):
        '파일 저장경로 생성'
        return os.path.join(self._img_directory_pre, self._cctvid + '_' + str(timestamp) + '.jpg')
    
    def generate_bulk_data(self, count=100):
        'bulk data 생성'
        dict_ = {column: [] for column in columns}
        # width, height 생성
        width, height = self._generate_width_height(new=True)
        # cctv 정보 생성
        cctv_id, cctv_name, center_name = self._generate_region_and_name(new=True)
        # # img_directory_pre 생성
        # img_directory_pre = self._generate_img_directory_pre(new=True)
        for _ in range(count):
            # columns = ['bboxes', 'scores', 'labels', 'timestamp', 'width', 'height', 'cctvid', 'region_and_name', 'centername', 'img_directory']
            # x1,y1 생성
            x, y = self.generate_starting_point()
            # bbox 생성
            bbox = self.generate_bbox(x, y)
            # score 생성
            score = self.generate_score()
            # label 생성
            label = self.generate_label()
            # timestamp 생성
            timestamp = self.generate_timestamp()
            # img_directory 생성
            # img_directory = os.path.join(img_directory_pre, cctv_id + '_' + str(timestamp) + '.jpg')
            img_directory = str(timestamp) + '.jpg'
            dict_['cctvId'].append(cctv_id)
            dict_['cctvName'].append(cctv_name)
            dict_['centerName'].append(center_name)
            dict_['timestamp'].append(timestamp)
            dict_['xCenter'].append(bbox[0])
            dict_['yCenter'].append(bbox[1])
            dict_['bWidth'].append(bbox[2])
            dict_['bHeight'].append(bbox[3])
            dict_['iWidth'].append(width)
            dict_['iHeight'].append(height)
            dict_['score'].append(score)
            dict_['label'].append(label)
            dict_['imgName'].append(img_directory)
        return pd.DataFrame.from_dict(dict_)

generator = GenerateRandomData()
generator.generate_label()
generator._generate_region_and_name()
generator.generate_timestamp()

1693805322025

In [30]:
df = pd.DataFrame(columns=columns)

for _ in range(1000):
    df = pd.concat([df, generator.generate_bulk_data(100)])
df

,cctvId,cctvName,centerName,timestamp,xCenter,yCenter,bWidth,bHeight,iWidth,iHeight,score,label,imgName
0,L34049,서리실사거리,원주교통정보센터,1695344223193,0.634967,0.512837,0.068759,0.033161,1280,720,0.529761,21,1695344223193.jpg
1,L34049,서리실사거리,원주교통정보센터,1694764752322,0.785636,0.591393,0.069170,0.123387,1280,720,0.829048,21,1694764752322.jpg
2,L34049,서리실사거리,원주교통정보센터,1694951763687,0.090698,0.472550,0.027584,0.098229,1280,720,0.721454,2,1694951763687.jpg
3,L34049,서리실사거리,원주교통정보센터,1694941332914,0.668762,0.740082,0.012650,0.017041,1280,720,0.784053,22,1694941332914.jpg
4,L34049,서리실사거리,원주교통정보센터,1693865436043,0.753576,0.354163,0.026733,0.106191,1280,720,0.891669,4,1693865436043.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,L020025,까치말사거리,인천교통정보센터,1695148994959,0.050586,0.726061,0.059331,0.026619,1600,900,0.601750,3,1695148994959.jpg
96,L020025,까치말사거리,인천교통정보센터,1694463612430,0.862640,0.390286,0.016900,0.052597,1600,900,0.716450,0,1694463612430.jpg
97,L020025,까치말사거리,인천교통정보센터,1695136209589,0.125310,0.910161,0.032045,0.078678,1600,900,0.738593,19,1695136209589.jpg
98,L020025,까치말사거리,인천교통정보센터,1693764664808,0.737901,0.188392,0.013601,0.019106,1600,900,0.547501,20,1693764664808.jpg


In [31]:
df.to_csv('generated_data6.csv', index=False)

In [32]:
df_ = pd.read_csv('generated_data6.csv')
df_

,cctvId,cctvName,centerName,timestamp,xCenter,yCenter,bWidth,bHeight,iWidth,iHeight,score,label,imgName
0,L34049,서리실사거리,원주교통정보센터,1695344223193,0.634967,0.512837,0.068759,0.033161,1280,720,0.529761,21,1695344223193.jpg
1,L34049,서리실사거리,원주교통정보센터,1694764752322,0.785636,0.591393,0.069170,0.123387,1280,720,0.829048,21,1694764752322.jpg
2,L34049,서리실사거리,원주교통정보센터,1694951763687,0.090698,0.472550,0.027584,0.098229,1280,720,0.721454,2,1694951763687.jpg
3,L34049,서리실사거리,원주교통정보센터,1694941332914,0.668762,0.740082,0.012650,0.017041,1280,720,0.784053,22,1694941332914.jpg
4,L34049,서리실사거리,원주교통정보센터,1693865436043,0.753576,0.354163,0.026733,0.106191,1280,720,0.891669,4,1693865436043.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,L020025,까치말사거리,인천교통정보센터,1695148994959,0.050586,0.726061,0.059331,0.026619,1600,900,0.601750,3,1695148994959.jpg
99996,L020025,까치말사거리,인천교통정보센터,1694463612430,0.862640,0.390286,0.016900,0.052597,1600,900,0.716450,0,1694463612430.jpg
99997,L020025,까치말사거리,인천교통정보센터,1695136209589,0.125310,0.910161,0.032045,0.078678,1600,900,0.738593,19,1695136209589.jpg
99998,L020025,까치말사거리,인천교통정보센터,1693764664808,0.737901,0.188392,0.013601,0.019106,1600,900,0.547501,20,1693764664808.jpg
